# Workflow for Marine Litter Targets Identification from Bathymetry        
#### ArcGis PRO with Spatial Analysis is required

**<span style='color:green'>Import necessary libraries</span>**

In [1]:
# Importing arcpy library
import arcpy
from arcpy import env
from arcpy.sa import *
import shutil
import os
# Check out any necessary licenses
arcpy.CheckOutExtension("spatial")
# Allow to overwrite existing output
arcpy.env.overwriteOutput = True

**<span style='color:green'>Define some inputs</span>**

In [2]:
# Define input, output and temporary folders (temporary folder will be deleted at the end)
#FolderInp must exist, FolderOut and FolderTemp will be created if not existing
FolderInp = "d:/MarGnet/tool/tool/inp/"
FolderOut = "d:/MarGnet/tool/tool/out/"
FolderTemp = "d:/MarGnet/tool/tool/temp/"
# Define input raster and output name
rasterInput = "test_2021.tif"
shapeOutput = "OBJ_2021.shp"
#_______________________________
# Define some parameters used in Arcgis toolboxes
AreaLimit = "0.25"
neighborhood = NbrRectangle(3, 3, "CELL")
remap = RemapRange([[-100,-.02,1],[-0.02,0,0]]) #[[-100,-.02,1],[-0.02,0,0]]
reclassField = "Value"

**<span style='color:green'>Run next cells recursively</span>**

In [3]:
#check folders
# ...temporary folder
if os.path.isdir(FolderTemp) == False:
    os.mkdir(FolderTemp)
# ... output folder
if os.path.isdir(FolderOut) == False:
    os.mkdir(FolderOut)
# ... input folder
error = False
if os.path.isdir(FolderInp) == False:
    print('Error...input folder is missing')
    error = True
if os.path.exists(FolderInp + rasterInput) == False:
    print('Error...input raster is missing')
    error = True

if error == False:
    # defining some local variables
    InputRaster0 = FolderInp + rasterInput
    InputRaster = FolderTemp + "negate"
    S1_Focal = FolderTemp + "s1_focal"
    S2_Fill = FolderTemp + "s2_fill"
    S3_Minus = FolderTemp + "s3_minus"
    S4_Reclass = FolderTemp + "S4_Reclass"
    S5_ToPolygons_shp = FolderTemp + "S4_Reclass.shp"
    S5_ToPolygons_shp__2_ = FolderTemp + "S4_Reclass_2.shp"
    S6_Poly_Filtered_shp = FolderTemp + "S6_Filtered.shp"
    S7_MinimumBoundary_shp = FolderOut + shapeOutput    
    
    print('check done')
else:
    print('check input folder/file')

check done


In [4]:
# Start workflow
print("1 - checking values...")
arcpy.CalculateStatistics_management(InputRaster0)
rast = arcpy.Raster (InputRaster0)
mi = rast.minimum
del rast
if mi < 0:
    print("values checked")
    print("1a - applying Negate...")
    # Process: Negate
    outNegate = Negate(InputRaster0)
    outNegate.save(InputRaster)
    del outNegate
    print("Negate applied")
    # Process: Focal Statistics
    print("2 - applying Focal Statistics...")
    outFocalStatistics = FocalStatistics(InputRaster, neighborhood, "MEAN","DATA")
    print("Focal Statistics applied")
else:
    # Process: Focal Statistics
    print("2 - applying Focal Statistics...")
    outFocalStatistics = FocalStatistics(InputRaster0, neighborhood, "MEAN","DATA")
    print("Focal Statistics applied")

outFocalStatistics.save(S1_Focal)
del outFocalStatistics
# Process: Fill
print("3 - applying Fill...")
outFill = Fill(S1_Focal, "")
outFill.save(S2_Fill)
del outFill
print("Fill applied")
# Process: Minus
print("4 - applying Minus...")
outMinus = Minus(S1_Focal, S2_Fill)
outMinus.save(S3_Minus)
del outMinus
print("Minus applied")
# Process: Reclassify
print("5 - applying Reclassify...")
outReclassify = Reclassify(S3_Minus, reclassField, remap, "NODATA")
    outReclassify.save(S4_Reclass)
del outReclassify
print("Reclassify applied")
# Process: Raster to Polygon
print("6 - applying Raster To Polygon...")
arcpy.RasterToPolygon_conversion(S4_Reclass, S5_ToPolygons_shp, "SIMPLIFY", "VALUE")
print("Raster To Polygon applied")
print("7 - classifying objects")
# Process: Add Geometry Attributes
print('     .adding Geometry Attributes')
arcpy.AddGeometryAttributes_management(S5_ToPolygons_shp, "AREA", "", "", "")
# Process: Select_Class_1_Area_025
print('     .filtering for area')
arcpy.Select_analysis(S5_ToPolygons_shp, S6_Poly_Filtered_shp, "GRIDCODE = 1 AND POLY_AREA >= " + AreaLimit)
# Process: Minimum Bounding Geometry
print('     .calculating Minimum Bounding Geometry')
arcpy.MinimumBoundingGeometry_management(S6_Poly_Filtered_shp, S7_MinimumBoundary_shp, "RECTANGLE_BY_WIDTH", "NONE", "", "MBG_FIELDS")
# Process: Delete Field
print('     .classifying objects')
arcpy.DeleteField_management(S7_MinimumBoundary_shp, "POLY_AREA")
# Process: Add Field_Ratio
#print('   .')
arcpy.AddField_management(S7_MinimumBoundary_shp, "ratio", "DOUBLE", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")
# Process: Calculate Field
#print('   .')
arcpy.CalculateField_management(S7_MinimumBoundary_shp, "ratio", "CalcRatio(!MBG_Length!,!MBG_Width!)","PYTHON_9.3","def CalcRatio(MBG_Length,MBG_Width): \r\n return MBG_Length/MBG_Width")
#Process: Add Field_Type
#print('   .classifying objects')
arcpy.CalculateField_management(S7_MinimumBoundary_shp, "Type", "Reclass(!ratio! )", "PYTHON_9.3", "def Reclass(ratio):\r\n  if (ratio <= 1.35):\r\n    return \"Tyre\"\r\n  elif (ratio >= 4.3):\r\n    return \"Poles\"\r\n  else:\r\n    return \"Others\"\r\n")
print("Done")

1 - checking values...
values checked
1a - applying Negate...
Negate applied
2 - applying Focal Statistics...
Focal Statistics applied
3 - applying Fill...
Fill applied
4 - applying Minus...
Minus applied
5 - applying Reclassify...
Reclassify applied
6 - applying Raster To Polygon...
Raster To Polygon applied
7 - classifying objects
     .adding Geometry Attributes
     .filtering for area
     .calculating Minimum Bounding Geometry
     .classifying objects
Done


In [ ]:
# Remove the temporary folder
shutil.rmtree(FolderTemp)
print('Temporary folder deleted')

In [ ]:
out_raster = arcpy.sa.Reclassify("04_minus", "Value", "-0.635702 -0.020000 1;-0.020000 0 0", "NODATA"); out_raster.save(r"D:\MarGnet\tool\tool\FromGis\04_clas")